In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
from nltk import sent_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from tqdm import tqdm
import random

In [2]:
dataset = pd.read_csv('papers_dataset.csv')
dataset.keywords = dataset.keywords.apply(ast.literal_eval)
dataset.sections = dataset.sections.apply(ast.literal_eval)

In [45]:
dataset.shape

(510, 13)

In [40]:
'''
random.seed(123)
categories = list(dataset.category.value_counts().keys())
test_set = set()
for i in categories:
    paper_titles = dataset[dataset.category == i].title.values
    test_set = test_set.union(set(random.sample(list(paper_titles), 3)))
    
dataset['partition'] = ['test' if i in test_set else 'train' for i in dataset.title.values]
dataset['paper_id'] = [f'paper_{i}' for i in range(dataset.shape[0])]
cols = list(dataset.columns[-1:]) + list(dataset.columns[:-1])
dataset = dataset[cols]
dataset.to_csv('papers_dataset.csv', index = False)
'''

In [11]:
all_sentences = []
categories = []

for row in tqdm(dataset.iterrows()):
    cat = row[1]['category']
    abstract_sents = sent_tokenize(row[1]['abstract'])
    all_sentences += abstract_sents
    categories += [cat] * len(abstract_sents)
    
    sections = list(row[1]['sections'].values())
    for section in sections:
        section_sents = sent_tokenize(section)
        all_sentences += section_sents
        categories += [cat] * len(section_sents)

510it [00:07, 71.90it/s]


In [14]:
categ_id = dict(zip(list(set(categories)), np.arange(len(list(set(categories))))))
sent_df = pd.DataFrame({'sentence': all_sentences, 'category': categories})
sent_df['categ_id'] = [categ_id[i] for i in sent_df.category.values]

## Read Embeddings

In [17]:
data_parts = []
for i in tqdm(range(5)):
    data = pd.read_csv(f'sentence_embeddings_{(i+1)*100}.csv')
    data_parts.append(data)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:25<00:00,  5.11s/it]


In [19]:
data_df = pd.concat(data_parts, axis = 0)
data_df.sent_tr_emb = data_df.sent_tr_emb.apply(lambda x: list(np.fromstring(x[1:-1],sep=' ').astype(float)))
data_df['categ_id'] = [categ_id[i] for i in data_df.category.values]

In [20]:
data_df.shape

(68777, 5)

In [24]:
embeddings = np.array(list(data_df.sent_tr_emb.values))